[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **YouTube Video Summarizer Agent**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

This Jupyter notebook implements an AI-powered YouTube video summarizer using CrewAI and Google's Gemini model. The system extracts transcripts from YouTube videos and generates comprehensive summaries through specialized AI agents.

## Dependencies

This cell imports all the required libraries and modules for the project:
- Basic Python libraries: `re`, `os`, `typing`
- Environment management: `python-dotenv`
- CrewAI framework: Core components for AI agent orchestration
- YouTube Transcript API: For extracting video transcripts
- Error handling classes for transcript-related exceptions

In [21]:
# !pip install crewai crewai[tools] youtube-transcript-api

In [22]:
# ================================================================================
# IMPORT DEPENDENCIES
# ================================================================================
"""
Import all necessary libraries for the project.
"""

import re
import os
from typing import Optional, List
from dotenv import load_dotenv

# CrewAI imports
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool

# YouTube Transcript API
from youtube_transcript_api import YouTubeTranscriptApi

# Error handling
try:
    from youtube_transcript_api._errors import (
        TranscriptsDisabled, 
        VideoUnavailable, 
        NoTranscriptFound
    )
except ImportError:
    # Fallback if specific errors aren't available
    TranscriptsDisabled = Exception
    VideoUnavailable = Exception
    NoTranscriptFound = Exception

print("✓ All dependencies imported successfully")


✓ All dependencies imported successfully


## Environment Configuration

This cell handles the setup of environment variables:
- Loads environment variables from `.env` file
- Retrieves and validates the Gemini API key
- Ensures the API key is properly configured before proceeding
- Displays confirmation messages for successful configuration

In [23]:

# ================================================================================
# ENVIRONMENT CONFIGURATION
# ================================================================================
"""
Load environment variables and validate API keys.
"""

# Load environment variables from .env file
load_dotenv()

# Retrieve API key
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Validate API key
if not GEMINI_API_KEY:
    raise ValueError(
        "❌ GEMINI_API_KEY not found in .env file\n"
        "Please create a .env file with: GEMINI_API_KEY=your_key_here"
    )

print("✓ Environment variables loaded")
print(f"✓ Gemini API key configured (length: {len(GEMINI_API_KEY)} chars)")


✓ Environment variables loaded
✓ Gemini API key configured (length: 39 chars)


## Utility Functions

This cell defines helper functions for YouTube video processing:
1. `get_video_id(youtube_url)`: 
   - Extracts the video ID from various YouTube URL formats
   - Supports standard watch URLs, short URLs, embed URLs, and YouTube Shorts
   - Returns the 11-character video ID or raises ValueError

2. `validate_video_id(video_id)`:
   - Validates the format of a YouTube video ID
   - Ensures ID matches the expected 11-character pattern
   - Returns boolean indicating validity

In [24]:

# ================================================================================
# UTILITY FUNCTIONS
# ================================================================================
"""
Helper functions for video ID extraction and validation.
"""

def get_video_id(youtube_url: str) -> str:
    """
    Extract YouTube video ID from various URL formats.
    
    Supports:
    - https://www.youtube.com/watch?v=VIDEO_ID
    - https://youtu.be/VIDEO_ID
    - https://www.youtube.com/embed/VIDEO_ID
    - https://www.youtube.com/shorts/VIDEO_ID
    
    Args:
        youtube_url: Full YouTube video URL
        
    Returns:
        11-character video ID
        
    Raises:
        ValueError: If URL format is invalid
    """
    regex_patterns = [
        r'(?:v=|\/videos\/|embed\/|\.be\/|shorts\/)([a-zA-Z0-9_-]{11})',
        r'youtube\.com\/watch\?v=([a-zA-Z0-9_-]{11})',
        r'youtu\.be\/([a-zA-Z0-9_-]{11})'
    ]
    
    for pattern in regex_patterns:
        match = re.search(pattern, youtube_url)
        if match:
            return match.group(1)
    
    raise ValueError(f"Invalid YouTube URL format: {youtube_url}")


def validate_video_id(video_id: str) -> bool:
    """
    Validate that a video ID has the correct format.
    
    Args:
        video_id: YouTube video ID to validate
        
    Returns:
        True if valid, False otherwise
    """
    return bool(re.match(r'^[a-zA-Z0-9_-]{11}$', video_id))


print("✓ Utility functions defined")


✓ Utility functions defined


## YouTube Transcript Extractor Tool

This cell implements the core transcript extraction functionality as a CrewAI tool:
- Uses YouTube Transcript API for reliable transcript extraction
- Implements a robust multi-step extraction process
- Supports multiple English language variants (CA, US, UK, auto-generated)
- Provides detailed error handling for common scenarios
- Includes comprehensive logging and statistics
- Returns formatted transcript text ready for analysis

Error cases handled:
- Disabled transcripts
- Unavailable videos
- Missing transcripts
- General API errors

In [25]:

# ================================================================================
# YOUTUBE TRANSCRIPT EXTRACTOR TOOL
# ================================================================================
"""
Create a CrewAI tool for extracting YouTube transcripts.

This uses Method 2 (the most reliable approach):
1. Create API instance
2. Get transcript list
3. Find matching language transcript
4. Fetch and extract text from snippets
"""

@tool("YouTube Transcript Extractor")
def fetch_youtube_transcript(youtube_url: str) -> str:
    """
    Fetches complete transcript from a YouTube video URL.
    
    This tool uses the reliable list() -> find_transcript() -> fetch() pattern
    from the YouTube Transcript API.
    
    Args:
        youtube_url: Full YouTube video URL (any format)
        
    Returns:
        Complete transcript text as a single string
        
    Error Handling:
        - Returns error message if video has no captions
        - Returns error message if video is unavailable
        - Returns error message for any other issues
    """
    try:
        # Step 1: Extract video ID from URL
        video_id = get_video_id(youtube_url)
        print(f"🔹 Processing video ID: {video_id}")
        
        # Step 2: Create API instance
        api = YouTubeTranscriptApi()
        
        # Step 3: Get list of available transcripts
        transcript_list = api.list(video_id)
        
        # Step 4: Find transcript in preferred languages
        # Tries languages in order: English (Canada), English (US), English (UK), English (auto)
        transcript = transcript_list.find_transcript([
            'en-CA',  # English (Canada)
            'en-US',  # English (US)
            'en-GB',  # English (UK)
            'en',     # English (auto-generated)
        ])
        
        print(f"   Selected: {transcript.language} ({transcript.language_code})")
        print(f"   Generated: {'Yes' if transcript.is_generated else 'No'}")
        
        # Step 5: Fetch the transcript data
        fetched_data = transcript.fetch()
        
        # Step 6: Extract text from snippets
        # FetchedTranscript contains 'snippets' - a list of FetchedTranscriptSnippet objects
        # Each snippet has: .text, .start, .duration attributes
        if hasattr(fetched_data, 'snippets'):
            snippets = fetched_data.snippets
        else:
            snippets = fetched_data
        
        # Join all snippet text into a single string
        transcript_text = " ".join([snippet.text for snippet in snippets])
        
        # Calculate statistics
        segment_count = len(list(snippets))
        word_count = len(transcript_text.split())
        
        print(f"✓ Extracted {len(transcript_text)} characters")
        print(f"✓ {segment_count} segments, ~{word_count} words\n")
        
        return transcript_text
        
    except TranscriptsDisabled:
        error_msg = "ERROR: This video does not have captions/subtitles enabled."
        print(f"❌ {error_msg}")
        return error_msg
        
    except VideoUnavailable:
        error_msg = "ERROR: Video is unavailable, private, or doesn't exist."
        print(f"❌ {error_msg}")
        return error_msg
        
    except NoTranscriptFound:
        error_msg = "ERROR: No transcript found in the requested languages (English)."
        print(f"❌ {error_msg}")
        return error_msg
        
    except Exception as e:
        error_msg = f"ERROR: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg


print("✓ YouTube transcript extraction tool created")


✓ YouTube transcript extraction tool created


## Gemini LLM Configuration

This cell configures the Google Gemini language model for use with CrewAI:
- Uses the efficient gemini-2.5-flash model
- Sets temperature to 0.3 for focused, consistent outputs
- Initializes with API key from environment
- Prepares model for agent-based summarization tasks

In [26]:

# ================================================================================
# CONFIGURE GEMINI LLM
# ================================================================================
"""
Configure the Google Gemini language model for use with CrewAI.
"""

# Initialize Gemini LLM with optimal settings
gemini_llm = LLM(
    model="gemini/gemini-2.5-flash",  # Fast and efficient model
    api_key=GEMINI_API_KEY,
    temperature=0.3,  # Lower temperature for more focused, consistent outputs
)

print("✓ Gemini LLM configured")
print("   Model: gemini-1.5-flash")
print("   Temperature: 0.3 (focused output)")


✓ Gemini LLM configured
   Model: gemini-1.5-flash
   Temperature: 0.3 (focused output)


## Agent Definitions

This cell creates two specialized AI agents for the workflow:

1. **Transcript Extractor Agent**:
   - Role: YouTube Transcript Extraction Specialist
   - Purpose: Extract accurate transcripts using provided tool
   - Features: Error handling, language selection, verification
   - Tools: fetch_youtube_transcript
   - No delegation allowed

2. **Content Summarizer Agent**:
   - Role: Content Analysis Expert
   - Purpose: Create comprehensive, structured summaries
   - Skills: Theme identification, key point extraction
   - Output: Reader-friendly, well-organized content
   - No delegation allowed

Both agents use the configured Gemini LLM with verbose output enabled.

In [27]:

# ================================================================================
# DEFINE AGENTS
# ================================================================================
"""
Create specialized AI agents for different parts of the workflow.
"""

# Agent 1: Transcript Extractor
extractor_agent = Agent(
    role="YouTube Transcript Extraction Specialist",
    goal=(
        "Extract complete, accurate transcripts from YouTube videos using the "
        "fetch_youtube_transcript tool. Always use the exact URL provided."
    ),
    backstory=(
        "You are an expert data extraction specialist with deep knowledge of "
        "YouTube's transcript system. You know how to handle different video "
        "formats, language options, and error cases. You always verify that "
        "transcripts are successfully extracted before passing them forward."
    ),
    tools=[fetch_youtube_transcript],
    llm=gemini_llm,
    verbose=True,
    allow_delegation=False,  # This agent should not delegate its work
)

# Agent 2: Content Summarizer
summarizer_agent = Agent(
    role="Content Analysis and Summarization Expert",
    goal=(
        "Create clear, comprehensive, and well-structured summaries from video "
        "transcripts that capture the essence and key insights of the content."
    ),
    backstory=(
        "You are a senior content analyst with years of experience in distilling "
        "complex information into digestible summaries. You excel at identifying "
        "main themes, extracting key points, and presenting information in a "
        "logical, reader-friendly format. You understand the importance of "
        "maintaining accuracy while being concise."
    ),
    llm=gemini_llm,
    verbose=True,
    allow_delegation=False,
)

print("✓ Agents created successfully")
print("   1. Transcript Extractor Agent")
print("   2. Content Summarizer Agent")


✓ Agents created successfully
   1. Transcript Extractor Agent
   2. Content Summarizer Agent


## Task Definitions

This cell defines the specific tasks for each agent:

1. **Extract Task**:
   - Assigned to: Transcript Extractor Agent
   - Purpose: Extract complete video transcript
   - Input: YouTube URL
   - Process: Use fetch_youtube_transcript tool
   - Output: Complete transcript text
   - Error handling requirements included

2. **Summarize Task**:
   - Assigned to: Content Summarizer Agent
   - Purpose: Create comprehensive summary
   - Input: Transcript from Extract Task
   - Required sections:
     - Overview (2-3 sentences)
     - Key Points (4-6 bullets)
     - Detailed Insights (2-3 paragraphs)
     - Key Takeaways (3-4 bullets)
   - Format requirements and length limits specified

In [28]:

# ================================================================================
# DEFINE TASKS
# ================================================================================
"""
Create specific tasks for each agent to execute.
"""

# Task 1: Extract Transcript
extract_task = Task(
    description=(
        "Extract the complete transcript from the YouTube video at this URL: {youtube_url}\n\n"
        "INSTRUCTIONS:\n"
        "1. Use the fetch_youtube_transcript tool with the EXACT URL provided\n"
        "2. Ensure the entire transcript is retrieved\n"
        "3. Verify that the transcript contains actual content (not an error message)\n"
        "4. Pass the complete transcript to the next task\n\n"
        "If you encounter any errors, report them clearly."
    ),
    expected_output=(
        "The complete transcript text from the video, with all segments joined "
        "into a continuous text. The transcript should be ready for analysis."
    ),
    agent=extractor_agent,
)

# Task 2: Summarize Content
summarize_task = Task(
    description=(
        "Analyze the video transcript from the previous task and create a comprehensive, "
        "professional summary.\n\n"
        "YOUR SUMMARY MUST INCLUDE:\n\n"
        "1. **Overview** (2-3 sentences)\n"
        "   - What is the video about?\n"
        "   - What is the main topic or theme?\n"
        "   - Who is the target audience?\n\n"
        "2. **Key Points** (4-6 bullet points)\n"
        "   - Main ideas discussed\n"
        "   - Important concepts explained\n"
        "   - Significant insights or findings\n\n"
        "3. **Detailed Insights** (2-3 paragraphs)\n"
        "   - Deeper analysis of the content\n"
        "   - How ideas connect to each other\n"
        "   - Context and implications\n\n"
        "4. **Key Takeaways** (3-4 bullet points)\n"
        "   - Most important lessons\n"
        "   - Actionable insights\n"
        "   - What viewers should remember\n\n"
        "FORMATTING REQUIREMENTS:\n"
        "- Use clear section headers\n"
        "- Write in professional, engaging language\n"
        "- Keep total length to 250-350 words\n"
        "- Use bullet points for lists\n"
        "- Use paragraphs for detailed explanations\n\n"
        "IMPORTANT: Base your summary ONLY on the transcript content. "
        "Do not add external information or make assumptions."
    ),
    expected_output=(
        "A well-structured, comprehensive summary that includes:\n"
        "- Clear overview of the video topic\n"
        "- 4-6 key points from the content\n"
        "- Detailed insights (2-3 paragraphs)\n"
        "- 3-4 actionable takeaways\n"
        "Total length: 250-350 words"
    ),
    agent=summarizer_agent,
    context=[extract_task],  # Receives output from extract_task
)

print("✓ Tasks defined successfully")
print("   1. Extract Task: Retrieve video transcript")
print("   2. Summarize Task: Analyze and summarize content")


✓ Tasks defined successfully
   1. Extract Task: Retrieve video transcript
   2. Summarize Task: Analyze and summarize content


## Crew Assembly

This cell creates the main CrewAI workflow:
- Combines both agents (extractor and summarizer)
- Arranges tasks in sequential order
- Enables verbose logging for process tracking
- Prepares the crew for execution with proper task dependencies

In [29]:
# ================================================================================
# ASSEMBLE CREW
# ================================================================================
"""
Assemble all agents and tasks into a cohesive crew.
"""

video_summarizer_crew = Crew(
    agents=[extractor_agent, summarizer_agent],
    tasks=[extract_task, summarize_task],
    process=Process.sequential,  # Tasks execute one after another
    verbose=True,  # Show detailed execution logs
)

print("✓ Crew assembled successfully")



✓ Crew assembled successfully


## Main Execution

This cell contains the main execution logic:
- Demonstrates usage with a sample YouTube URL
- Executes the crew workflow with proper error handling
- Displays formatted results or error messages
- Shows execution status and completion indicators

Note: Replace the sample URL with your desired YouTube video URL when running the notebook.

In [30]:
if __name__ == "__main__":
    print("🚀 YouTube Video Summarizer Agent is ready to run!")
    
    youtube_url = "https://youtu.be/qU3Rc6_B9es?si=8GffTVLOfjKNjaEZ"
    
    try:
        # Run the crew with the URL as input
        result = video_summarizer_crew.kickoff(inputs={'youtube_url': youtube_url})
        
        # Display results
        print(f"\n{'='*80}")
        print(f"✅ SUMMARY COMPLETE")
        print(f"{'='*80}\n")
        print(result)
        print(f"\n{'='*80}\n")
        
    except Exception as e:
        print(f"❌ Error during execution: {e}")

🚀 YouTube Video Summarizer Agent is ready to run!
🔹 Processing video ID: qU3Rc6_B9es
   Selected: English (Canada) (en-CA)
   Generated: No
✓ Extracted 33492 characters
✓ 527 segments, ~6127 words


✅ SUMMARY COMPLETE

### Comprehensive Summary of Production Python Code Principles

**Overview**
This video serves as a comprehensive guide for Python developers aiming to write production-grade code, highlighting key design principles that differentiate senior practitioners. It delves into eight crucial concepts—from cohesion to simplicity—using in-depth examples to demonstrate how to build scalable, robust, and maintainable software systems. The target audience includes Python developers serious about improving their coding practices and advancing their skills.

**Key Points**
*   **Cohesion & Single Responsibility:** Each code unit should have one distinct purpose; related elements grouped for clarity.
*   **Encapsulation & Abstraction:** Hide internal details, exposing only necessary in